# Setting things up

In [1]:
!pip install diffcp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 25.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/locuslab/optnet.git

Cloning into 'optnet'...
remote: Enumerating objects: 162, done.
remote: Total 162 (delta 0), reused 0 (delta 0), pack-reused 162
Receiving objects: 100% (162/162), 1.48 MiB | 4.06 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [3]:
%cd optnet

/content/optnet


In [4]:
!git clone https://github.com/locuslab/qpth.git

Cloning into 'qpth'...
remote: Enumerating objects: 853, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 853 (delta 10), reused 10 (delta 3), pack-reused 827
Receiving objects: 100% (853/853), 330.00 KiB | 10.00 MiB/s, done.
Resolving deltas: 100% (512/512), done.


In [5]:
%cd qpth

/content/optnet/qpth


In [6]:
!pip install e .

Processing /content/optnet/qpth
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for e: filename=e-1.4.5-py3-none-any.whl size=2794 sha256=d4b04107805b4261292e4148d69bf207e7f71baf13fe058561b3ad58d1b680e9
  Stored in directory: /root/.cache/pip/wheels/3d/dc/3e/208e61209d7750644677d72934d2997177aa34e894bd398e16
  Created wheel for qpth: filename=qpth-0.0.16-py3-none-any.whl size=19522 sha256=f2df237d13e0f976fcda77ad7553fdc2cacd3a72baa106dae892b7785f1ff35f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z9yi1k4a/wheels/87/18/75/ab21aee042e161a25fa4ed66ae13ba5c82467c421905b68ea7
Successfully built e qpth


In [7]:
%cd /content/optnet/cls/

/content/optnet/cls


# Experiment 1: OptNet vs 2 fully-connected layers on FashionMNIST

DataLoaders

In [8]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
batchSz=64
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)
kwargs = {'num_workers': 1, 'pin_memory': True}
trainLoader = DataLoader(
            dset.FashionMNIST('data/Fmnist', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=batchSz, shuffle=True, **kwargs)
testLoader = DataLoader(
            dset.FashionMNIST('data/Fmnist', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))])))

100%|██████████| 26421880/26421880 [00:02<00:00, 11976646.69it/s]


Extracting data/Fmnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/Fmnist/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201344.87it/s]


Extracting data/Fmnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/Fmnist/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3850878.36it/s]


Extracting data/Fmnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/Fmnist/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7823288.77it/s]

Extracting data/Fmnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/Fmnist/FashionMNIST/raw



Train loop

In [9]:
from torch.autograd import Function, Variable
import torch.nn.functional as F
def train(epoch, net, trainLoader, optimizer, trainF):
    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    for batch_idx, (data, target) in enumerate(trainLoader):

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        # make_graph.save('/tmp/t.dot', loss.creator); assert(False)
        loss.backward()
        optimizer.step()
        nProcessed += len(data)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect = pred.ne(target.data).cpu().sum()
        err = 100.*incorrect/len(data)
        partialEpoch = epoch + batch_idx / len(trainLoader) - 1
        print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
            partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
            loss.item(), err))

        trainF.write('{},{},{}\n'.format(partialEpoch, loss.item(), err))
        trainF.flush()

Test loop

In [10]:
import torch.nn.functional as F
def test(epoch, net, testLoader, optimizer, testF):
    net.eval()
    test_loss = 0
    incorrect = 0
    for data, target in testLoader:

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.nll_loss(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect += pred.ne(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader.dataset)
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))

    testF.write('{},{},{}\n'.format(epoch, test_loss, err))
    testF.flush()

Defining the OptNet model

In [11]:
import models
nineq=10
nHidden=500
bn=True
net = models.OptNet(28*28, nHidden, 10, bn, nineq)
net = net.cuda()
print(net)

OptNet(
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [12]:
import torch.optim as optim
params = list(net.parameters())
optimizer = optim.Adam(params, weight_decay=1e-4)

In [13]:
trainF = open('train.csv', 'w')
testF = open('test.csv', 'w')

Training OptNet (this takes ~ 30 minutes)

In [14]:
import torch
import os
nEpoch=5
for epoch in range(1,nEpoch + 1):
        train(epoch, net, trainLoader, optimizer, trainF)
        test(epoch, net, testLoader, optimizer, testF)
        try:
            torch.save(net, os.path.join("/content/", 'latest.pth'))
        except:
            pass
        os.system('./plot.py "{}" &'.format("/content/"))

trainF.close()
testF.close()

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:738: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1992.)
  LU, pivots, infos = torch._lu_with_info(
/usr/local/lib/python3.10/dist-packages/qpth/solvers/pdipm/batch.py:395: UserWarning: torch.lu_solve is deprecated in favor of torch.linalg.lu_solveand will be removed in a future PyTorch release.
Note that torch.linalg.lu_solve has its arguments reversed.
X = torch.lu_solve(B, LU, pivots)
should be replaced with
X = torch.linalg.lu_solve(LU, pivots, B) (Triggered internally at ../aten/src/

Train Epoch: 0.00 [64/60000 (0%)]	Loss: 8.140724	Error: 95.312500
Train Epoch: 0.00 [128/60000 (0%)]	Loss: 4.719702	Error: 81.250000
Train Epoch: 0.00 [192/60000 (0%)]	Loss: 4.584157	Error: 78.125000
Train Epoch: 0.00 [256/60000 (0%)]	Loss: 3.480546	Error: 84.375000
Train Epoch: 0.00 [320/60000 (0%)]	Loss: 3.384925	Error: 78.125000
Train Epoch: 0.01 [384/60000 (1%)]	Loss: 3.091430	Error: 76.562500
Train Epoch: 0.01 [448/60000 (1%)]	Loss: 2.688430	Error: 84.375000
Train Epoch: 0.01 [512/60000 (1%)]	Loss: 2.627575	Error: 76.562500
Train Epoch: 0.01 [576/60000 (1%)]	Loss: 1.947255	Error: 64.062500
Train Epoch: 0.01 [640/60000 (1%)]	Loss: 2.131766	Error: 64.062500
Train Epoch: 0.01 [704/60000 (1%)]	Loss: 3.162570	Error: 82.812500
Train Epoch: 0.01 [768/60000 (1%)]	Loss: 2.199957	Error: 75.000000
Train Epoch: 0.01 [832/60000 (1%)]	Loss: 1.948932	Error: 67.187500
Train Epoch: 0.01 [896/60000 (1%)]	Loss: 2.359564	Error: 71.875000
Train Epoch: 0.01 [960/60000 (1%)]	Loss: 2.025000	Error: 68.750

<ipython-input-10-0dbb4f553204>:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.5842, Error: 1864/10000 (19%)

Train Epoch: 1.00 [64/60000 (0%)]	Loss: 0.440063	Error: 10.937500
Train Epoch: 1.00 [128/60000 (0%)]	Loss: 0.403863	Error: 15.625000
Train Epoch: 1.00 [192/60000 (0%)]	Loss: 0.693099	Error: 26.562500
Train Epoch: 1.00 [256/60000 (0%)]	Loss: 0.525086	Error: 15.625000
Train Epoch: 1.00 [320/60000 (0%)]	Loss: 0.495288	Error: 17.187500
Train Epoch: 1.01 [384/60000 (1%)]	Loss: 0.606750	Error: 20.312500
Train Epoch: 1.01 [448/60000 (1%)]	Loss: 0.600722	Error: 14.062500
Train Epoch: 1.01 [512/60000 (1%)]	Loss: 0.363353	Error: 12.500000
Train Epoch: 1.01 [576/60000 (1%)]	Loss: 0.750456	Error: 20.312500
Train Epoch: 1.01 [640/60000 (1%)]	Loss: 0.547136	Error: 21.875000
Train Epoch: 1.01 [704/60000 (1%)]	Loss: 0.441408	Error: 14.062500
Train Epoch: 1.01 [768/60000 (1%)]	Loss: 0.340199	Error: 10.937500
Train Epoch: 1.01 [832/60000 (1%)]	Loss: 0.724260	Error: 21.875000
Train Epoch: 1.01 [896/60000 (1%)]	Loss: 0.568413	Error: 18.750000
Train

In [15]:
!python plot.py ./

Created ./loss.png
Created ./error.png
sh: 1: convert: not found
Created ./loss-error.png


Error of OptNet

In [16]:
from PIL import Image

im = Image.open('error.png')
im.show()

Loss of OptNet

In [17]:
im = Image.open('loss.png')
im.show()

In [18]:
#resetting the plots
trainF.close()
testF.close()
%rm error.png
%rm loss.png
%rm train.csv
%rm test.csv

Defining the fully-connected model

In [19]:
net2 = models.FC(nHidden, bn)
trainF = open('train.csv', 'w')
testF = open('test.csv', 'w')
net2.cuda()

FC(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
)

In [20]:
import torch.optim as optim
params = list(net2.parameters())
optimizer = optim.Adam(params, weight_decay=1e-4)

Training the FC model

In [21]:
import torch
import os
nEpoch=5
for epoch in range(1,nEpoch + 1):
        train(epoch, net2, trainLoader, optimizer, trainF)
        test(epoch, net2, testLoader, optimizer, testF)
        try:
            torch.save(net, os.path.join("/content/", 'latest.pth'))
        except:
            pass
        os.system('./plot.py "{}" &'.format("/content/"))

trainF.close()
testF.close()

/content/optnet/cls/models.py:118: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0.00 [64/60000 (0%)]	Loss: 2.308137	Error: 89.062500
Train Epoch: 0.00 [128/60000 (0%)]	Loss: 1.984495	Error: 71.875000
Train Epoch: 0.00 [192/60000 (0%)]	Loss: 1.851730	Error: 59.375000
Train Epoch: 0.00 [256/60000 (0%)]	Loss: 1.707437	Error: 51.562500
Train Epoch: 0.00 [320/60000 (0%)]	Loss: 1.663163	Error: 45.312500
Train Epoch: 0.01 [384/60000 (1%)]	Loss: 1.749678	Error: 48.437500
Train Epoch: 0.01 [448/60000 (1%)]	Loss: 1.602918	Error: 35.937500
Train Epoch: 0.01 [512/60000 (1%)]	Loss: 1.563688	Error: 32.812500
Train Epoch: 0.01 [576/60000 (1%)]	Loss: 1.410724	Error: 20.312500
Train Epoch: 0.01 [640/60000 (1%)]	Loss: 1.506888	Error: 29.687500
Train Epoch: 0.01 [704/60000 (1%)]	Loss: 1.476650	Error: 25.000000
Train Epoch: 0.01 [768/60000 (1%)]	Loss: 1.567808	Error: 40.625000
Train Epoch: 0.01 [832/60000 (1%)]	Loss: 1.601397	Error: 40.625000
Train Epoch: 0.01 [896/60000 (1%)]	Loss: 1.589222	Error: 43.750000
Train Epoch: 0.01 [960/60000 (1%)]	Loss: 1.463386	Error: 31.250

<ipython-input-10-0dbb4f553204>:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.4562, Error: 1589/10000 (16%)

Train Epoch: 1.00 [64/60000 (0%)]	Loss: 0.524746	Error: 20.312500
Train Epoch: 1.00 [128/60000 (0%)]	Loss: 0.427814	Error: 20.312500
Train Epoch: 1.00 [192/60000 (0%)]	Loss: 0.374523	Error: 14.062500
Train Epoch: 1.00 [256/60000 (0%)]	Loss: 0.362753	Error: 14.062500
Train Epoch: 1.00 [320/60000 (0%)]	Loss: 0.352339	Error: 15.625000
Train Epoch: 1.01 [384/60000 (1%)]	Loss: 0.352994	Error: 9.375000
Train Epoch: 1.01 [448/60000 (1%)]	Loss: 0.310709	Error: 9.375000
Train Epoch: 1.01 [512/60000 (1%)]	Loss: 0.459476	Error: 18.750000
Train Epoch: 1.01 [576/60000 (1%)]	Loss: 0.305154	Error: 12.500000
Train Epoch: 1.01 [640/60000 (1%)]	Loss: 0.467852	Error: 18.750000
Train Epoch: 1.01 [704/60000 (1%)]	Loss: 0.258998	Error: 6.250000
Train Epoch: 1.01 [768/60000 (1%)]	Loss: 0.375777	Error: 12.500000
Train Epoch: 1.01 [832/60000 (1%)]	Loss: 0.417453	Error: 17.187500
Train Epoch: 1.01 [896/60000 (1%)]	Loss: 0.603063	Error: 21.875000
Train Ep

In [22]:
!python plot.py ./

Created ./loss.png
Created ./error.png
sh: 1: convert: not found
Created ./loss-error.png


Error

In [23]:
im = Image.open('error.png')
im.show()

Loss

In [24]:
im = Image.open('loss.png')
im.show()

In [25]:
nparams1 = sum([p.data.nelement() for p in net.parameters()])
nparams2 = sum([p.data.nelement() for p in net2.parameters()])
print("OptNet parameters: ",nparams1)
print("FC parameters: ",nparams2)

OptNet parameters:  398750
FC parameters:  398640


**Conclusion**
The results are consistent with the paper's, the OptNet architecture performs approximatively as well as a fullyconnected architecture. The main drawback of the OptNet is the extremely slow training process, which was very noticeable during this experiment.

In [26]:
trainF.close()
testF.close()
%rm error.png
%rm loss.png
%rm train.csv
%rm test.csv

# Experiment 2: Using conic solver in OptNet

(this experiment didnt work, it was too hard to adapt the solvers to eachother)

In [27]:
import torch

import torch.nn as nn
from torch.autograd import Function, Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F

import diffcp

class OptNet(nn.Module):
    def __init__(self, nFeatures, nHidden, nCls, bn, nineq=200, neq=0, eps=1e-4):
        super().__init__()

        self.nFeatures = nFeatures
        self.nHidden = nHidden
        self.bn = bn
        self.nCls = nCls

        if bn:
            self.bn1 = nn.BatchNorm1d(nHidden)
            self.bn2 = nn.BatchNorm1d(nCls)

        self.fc1 = nn.Linear(nFeatures, nHidden)
        self.fc2 = nn.Linear(nHidden, nCls)

        # self.qp_z0 = nn.Linear(nCls, nCls)
        # self.qp_s0 = nn.Linear(nCls, nineq)

        assert(neq==0)
        self.M = Variable(torch.tril(torch.ones(nCls, nCls)).cuda())
        self.L = Parameter(torch.tril(torch.rand(nCls, nCls).cuda()))
        self.G = Parameter(torch.Tensor(nineq,nCls).uniform_(-1,1).cuda())
        self.z0 = Parameter(torch.zeros(nCls).cuda())
        self.s0 = Parameter(torch.ones(nineq).cuda())

        self.nineq = nineq
        self.neq = neq
        self.eps = eps

    def forward(self, x):
        nBatch = x.size(0)

        # FC-ReLU-(BN)-FC-ReLU-(BN)-QP-Softmax
        x = x.view(nBatch, -1)
        x = F.relu(self.fc1(x))
        if self.bn:
            x = self.bn1(x)
        x = F.relu(self.fc2(x))
        if self.bn:
            x = self.bn2(x)

        L = self.M*self.L
        Q = L.mm(L.t()) + self.eps*Variable(torch.eye(self.nCls)).cuda()
        Q = Q.unsqueeze(0).expand(nBatch, self.nCls, self.nCls)
        G = self.G.unsqueeze(0).expand(nBatch, self.nineq, self.nCls)
        # z0 = self.qp_z0(x)
        # s0 = self.qp_s0(x)
        z0 = self.z0.unsqueeze(0).expand(nBatch, self.nCls)
        s0 = self.s0.unsqueeze(0).expand(nBatch, self.nineq)
        h = z0.mm(self.G.t())+s0
        e = Variable(torch.Tensor())
        inputs = x
        # Define cone K
        K = diffcp.Cone(["SOC", "EQ"])

        # Define problem data for DiffCP
        c = torch.mv(Q, torch.Tensor([1., 1.]))
        A = torch.cat([G, torch.eye(2)], dim=0)
        b = torch.cat([h, torch.Tensor([1., 1.])], dim=0)
        x, y, s, D, DT = diffcp.solve_and_derivative(A, b, c, K)
        x = x.float()
        # x = x[:,:10].float()

        return F.log_softmax(x)

In [28]:
nineq=10
nHidden=500
bn=True
net = OptNet(28*28, nHidden, 10, bn, nineq)
net = net.cuda()
print(net)

OptNet(
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [29]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
batchSz=64
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)
kwargs = {'num_workers': 1, 'pin_memory': True}
trainLoader = DataLoader(
            dset.FashionMNIST('data/Fmnist', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=batchSz, shuffle=True, **kwargs)
testLoader = DataLoader(
            dset.FashionMNIST('data/Fmnist', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))])))

In [30]:
from torch.autograd import Function, Variable
import torch.nn.functional as F
def train(epoch, net, trainLoader, optimizer, trainF):
    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    for batch_idx, (data, target) in enumerate(trainLoader):

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        # make_graph.save('/tmp/t.dot', loss.creator); assert(False)
        loss.backward()
        optimizer.step()
        nProcessed += len(data)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect = pred.ne(target.data).cpu().sum()
        err = 100.*incorrect/len(data)
        partialEpoch = epoch + batch_idx / len(trainLoader) - 1
        print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
            partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
            loss.item(), err))

        trainF.write('{},{},{}\n'.format(partialEpoch, loss.item(), err))
        trainF.flush()

In [31]:
import torch.nn.functional as F
def test(epoch, net, testLoader, optimizer, testF):
    net.eval()
    test_loss = 0
    incorrect = 0
    for data, target in testLoader:

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.nll_loss(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect += pred.ne(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader.dataset)
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))

    testF.write('{},{},{}\n'.format(epoch, test_loss, err))
    testF.flush()

In [32]:
import torch.optim as optim
params = list(net.parameters())
optimizer = optim.Adam(params, weight_decay=1e-4)

In [33]:
trainF = open('train.csv', 'w')
testF = open('test.csv', 'w')

In [34]:
import torch
import os
nEpoch=5
for epoch in range(1,nEpoch + 1):
        train(epoch, net, trainLoader, optimizer, trainF)
        test(epoch, net, testLoader, optimizer, testF)
        try:
            torch.save(net, os.path.join("/content/", 'latest.pth'))
        except:
            pass
        os.system('./plot.py "{}" &'.format("/content/"))

trainF.close()
testF.close()

AttributeError: module 'diffcp' has no attribute 'Cone'